# Collecting Reddit Data from r/WallStreetBets

## Part 1: Setup

### Install Packages

In [13]:
# Install the packages if they don't exist.
%pip install praw
%pip install pyarrow
%pip install python-dotenv

# Import Packages
import praw, time, os, pyarrow
from IPython.display import display
from dotenv import load_dotenv, dotenv_values
from requests import Session
import pandas as pd
from IPython import get_ipython

# Load environment variables from .env file
load_dotenv('.env')
config = dotenv_values()

# Get config from colab or other environment.
def is_colab():
    return get_ipython().__class__.__module__ == "google.colab._shell"

if is_colab():
    from google.colab import userdata
    config = {}
    config['CLIENT_SECRET'] = userdata.get('CLIENT_SECRET')
    config['CLIENT_ID'] = userdata.get('CLIENT_ID')
    config['NAME'] = userdata.get('NAME')
    config['REDIRECT_URI'] = userdata.get('REDIRECT_URI')
    config['USERNAME'] = userdata.get('USERNAME')
    config['PASSWORD'] = userdata.get('PASSWORD')

else:
    load_dotenv('.env')
    config = dotenv_values()


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Part 2: Collecting Submissions from Reddit

### Open Reddit Connection

In [14]:
# Create a custom session with a timeout
session = Session()
session.headers.update({'User-Agent': 'praw'})
session.timeout = 10  # Set a timeout of 10 seconds

# Login to Reddit using PRAW
reddit = praw.Reddit(
    client_id=config['CLIENT_ID'],
    client_secret=config['CLIENT_SECRET'],
    requestor_kwargs={"session": session},
    username=config['USERNAME'],
    password=config['PASSWORD'],
    user_agent="CS470 ML Project Access by u/GregorybLafayetteML"
)

# Add some peripheral config data
reddit.config.log_requests = 1
reddit.config.store_json_result = True

# Test the connection
try:
    username = reddit.user.me()
    print("Successfully logged in to Reddit!")
    print(f"Logged in as: u/{username}")
except Exception as e:
    print(f"Failed to log in: {e}")

Successfully logged in to Reddit!
Logged in as: u/GregorybLafayetteML


### Accessing Reddit Data

To access reddit posts, we'll need send a request with the number of post we want to get. The following example finds the top 10 hottest posts on the u/wallstreetbets subreddit. We'll show the post title, score, flair, and URL.

In [15]:
top_posts = reddit.subreddit('wallstreetbets').hot(limit=10)
print("Top 10 hot posts from r/wallstreetbets:")
for post in top_posts:
    print(f"Title: {post.title}, Score: {post.score}, Flair: {post.link_flair_text}, URL: {post.url}")

Top 10 hot posts from r/wallstreetbets:
Title: Weekly Earnings Thread 4/14 - 4/18, Score: 208, Flair: Earnings Thread, URL: https://i.redd.it/b51iqmecn7ue1.jpeg
Title: What Are Your Moves Tomorrow, April 17, 2025, Score: 65, Flair: Daily Discussion, URL: https://www.reddit.com/r/wallstreetbets/comments/1k0tt5k/what_are_your_moves_tomorrow_april_17_2025/
Title: Powell indicates tariffs could pose a challenge for the Fed between controlling inflation and supporting economic growth, Score: 5035, Flair: News, URL: https://www.cnbc.com/2025/04/16/powell-indicates-tariffs-could-pose-a-two-pronged-policy-challenge-for-the-fed-.html
Title: Powell’s future…., Score: 1506, Flair: Meme, URL: https://v.redd.it/c0gaopbap8ve1
Title: Just doubled down on my bet 😬😬😬, Score: 1933, Flair: YOLO, URL: https://i.redd.it/kirarknsw7ve1.jpeg
Title: Retail sales surged in March as Americans rushed to beat Trump’s tariffs, Score: 2221, Flair: News, URL: https://www.reddit.com/r/wallstreetbets/comments/1k0jvyb/r

For this project, we'll need far more than ten posts at a time. The reddit API will limit our access to 100 posts at a time. Fortunately, the api uses a ListingGenerator which allows us to access our metered connection in sequential blocks. The following example shows how we can utilize this behavior, grabbing blocks of 100 posts at a time. In our example, we'll grab blocks of posts until we reach 5000 posts or our access times out. Notice that the procedure ends early with around 750-800 posts collected. The results are sparce, because our connection either timed out or was metered down by reddit. The latter option is more likely.

In [16]:
# Access the subreddit
subreddit = reddit.subreddit("wallstreetbets")

# Initialize variables
batch_size = 50 # Number of posts per batch
total_posts = 5000  # Total number of posts to fetch
all_posts = []  # To store all the retrieved posts
after = None  # To keep track of the last post for pagination

# Fetch posts in batches
while len(all_posts) < total_posts:
    # Fetch the next batch of posts
    submissions = subreddit.new(limit=batch_size, params={"after": after})

    batch_posts = []
    for submission in submissions:
        batch_posts.append(submission)

        # Update the `after` variable with the last submission's fullname
        after = submission.fullname

    # Add the batch to the main list
    all_posts.extend(batch_posts)

    # Exit loop if no more posts are available
    if not batch_posts:
        print("No more posts to fetch.")
        break

    # Optional delay to avoid rate limits
    time.sleep(5)  # Adjust the delay as necessary

# Process the data (example: print the total number of posts fetched)
print(f"Fetched {len(all_posts)} posts in total.")

No more posts to fetch.
Fetched 797 posts in total.


Now that we have collected a large portion of posts/submssions, we'll parse the results and construct a dataframe with this data. We're going to collect more fields from this data than we might need right now, avoiding data limitations in the future.

In [17]:
# Parse are submission objects that we collected.
fields = ('title',
          'created_utc',
          'id',
          'is_original_content',
          'link_flair_text',
          'locked',
          'name',
          'num_comments',
          'over_18',
          'permalink',
          'selftext',
          'spoiler',
          'upvote_ratio')
list_of_submissions = []

# Parse each submission into a dictionary of the lised fields.
for submission in all_posts:
    full = vars(submission)
    sub_dict = {field:full[field] for field in fields}
    list_of_submissions.append(sub_dict)

# Create a python dataframe of these submissions.
collected_data = pd.DataFrame.from_records(list_of_submissions)

# Display the dataframe.
display(collected_data)

,title,created_utc,id,is_original_content,link_flair_text,locked,name,num_comments,over_18,permalink,selftext,spoiler,upvote_ratio
0,Powell to Volatile Stock Market: You’re on You...,1.744836e+09,1k0unbq,False,News,False,t3_1k0unbq,2,False,/r/wallstreetbets/comments/1k0unbq/powell_to_v...,,False,0.86
1,Super sick timing,1.744835e+09,1k0umm6,False,Loss,False,t3_1k0umm6,3,False,/r/wallstreetbets/comments/1k0umm6/super_sick_...,,False,0.75
2,My second week trading options. SPY BAC and BABA,1.744835e+09,1k0ugf8,False,Gain,False,t3_1k0ugf8,1,False,/r/wallstreetbets/comments/1k0ugf8/my_second_w...,This is my second—and hopefully last—week trad...,False,0.75
3,Don’t see this too often,1.744835e+09,1k0ubo1,False,Discussion,False,t3_1k0ubo1,11,False,/r/wallstreetbets/comments/1k0ubo1/dont_see_th...,,False,0.86
4,I was about to rope and then +97k,1.744834e+09,1k0u286,False,Gain,False,t3_1k0u286,10,False,/r/wallstreetbets/comments/1k0u286/i_was_about...,,False,0.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,Account Blew Up In One Day,1.743226e+09,1jmff4z,False,Loss,False,t3_1jmff4z,106,False,/r/wallstreetbets/comments/1jmff4z/account_ble...,Had a successful 20 day run up with a small $2...,False,0.95
793,Pretty Good Friday,1.743223e+09,1jmei0h,False,Gain,False,t3_1jmei0h,23,False,/r/wallstreetbets/comments/1jmei0h/pretty_good...,Had a feeling we’d sell off hard for PCE even ...,False,0.96
794,"Oracle to build cluster of 30,000 AMD MI355X GPUs",1.743214e+09,1jmbxap,False,News,False,t3_1jmbxap,50,False,/r/wallstreetbets/comments/1jmbxap/oracle_to_b...,,False,0.96
795,299 to 5k. Baby gains but fuck NVDA,1.743210e+09,1jmamu6,False,Gain,False,t3_1jmamu6,78,False,/r/wallstreetbets/comments/1jmamu6/299_to_5k_b...,,False,0.92


### Saving Reddit Data

In [18]:
# Save the collected data to parquet format
SUBMISSION_PARQUET_PATH = './data/wallstreetbets-collection.parquet'

# Create a pyarrow schema for the data types.
submission_schema = pyarrow.schema([
    ('title', pyarrow.string()),
    ('created_utc', pyarrow.float64()),
    ('id', pyarrow.string()),
    ('is_original_content', pyarrow.bool_()),
    ('link_flair_text', pyarrow.string()),
    ('locked', pyarrow.bool_()),
    ('name', pyarrow.string()),
    ('num_comments', pyarrow.int64()),
    ('over_18', pyarrow.bool_()),
    ('permalink', pyarrow.string()),
    ('selftext', pyarrow.string()),
    ('spoiler', pyarrow.bool_()),
    ('upvote_ratio', pyarrow.float64()),
])

In [19]:
# If the parqet does not exist, create it.
if not os.path.exists(SUBMISSION_PARQUET_PATH):
    collected_data.to_parquet(SUBMISSION_PARQUET_PATH, engine='pyarrow', schema=submission_schema)

# If the data file already exist, merge new data with the existing one.
else:
    old_parquet = pd.read_parquet(SUBMISSION_PARQUET_PATH, engine='pyarrow', schema=submission_schema)
    new_parquet = pd.concat([old_parquet, collected_data])
    new_parquet = new_parquet.drop_duplicates(subset=['id','title','created_utc','name','permalink'], keep='last').reset_index(drop=True)
    new_parquet.to_parquet(SUBMISSION_PARQUET_PATH, engine='pyarrow', schema=submission_schema)

# Use the new collected data to get comment stuff.
SUBMISSION_PARQUET_PATH = './data/wallstreetbets-collection.parquet'
submission_collection = pd.read_parquet(SUBMISSION_PARQUET_PATH, engine='pyarrow', schema=submission_schema)
display(submission_collection)

,title,created_utc,id,is_original_content,link_flair_text,locked,name,num_comments,over_18,permalink,selftext,spoiler,upvote_ratio
0,Nivea Along,1.744832e+09,1k0t4jk,False,YOLO,False,t3_1k0t4jk,5,False,/r/wallstreetbets/comments/1k0t4jk/nivea_along/,After -7% yesterday and -10% today,False,0.67
1,Powell to Volatile Stock Market: You’re on You...,1.744836e+09,1k0unbq,False,News,False,t3_1k0unbq,2,False,/r/wallstreetbets/comments/1k0unbq/powell_to_v...,,False,0.86
2,Super sick timing,1.744835e+09,1k0umm6,False,Loss,False,t3_1k0umm6,3,False,/r/wallstreetbets/comments/1k0umm6/super_sick_...,,False,0.75
3,My second week trading options. SPY BAC and BABA,1.744835e+09,1k0ugf8,False,Gain,False,t3_1k0ugf8,1,False,/r/wallstreetbets/comments/1k0ugf8/my_second_w...,This is my second—and hopefully last—week trad...,False,0.75
4,Don’t see this too often,1.744835e+09,1k0ubo1,False,Discussion,False,t3_1k0ubo1,11,False,/r/wallstreetbets/comments/1k0ubo1/dont_see_th...,,False,0.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,Account Blew Up In One Day,1.743226e+09,1jmff4z,False,Loss,False,t3_1jmff4z,106,False,/r/wallstreetbets/comments/1jmff4z/account_ble...,Had a successful 20 day run up with a small $2...,False,0.95
794,Pretty Good Friday,1.743223e+09,1jmei0h,False,Gain,False,t3_1jmei0h,23,False,/r/wallstreetbets/comments/1jmei0h/pretty_good...,Had a feeling we’d sell off hard for PCE even ...,False,0.96
795,"Oracle to build cluster of 30,000 AMD MI355X GPUs",1.743214e+09,1jmbxap,False,News,False,t3_1jmbxap,50,False,/r/wallstreetbets/comments/1jmbxap/oracle_to_b...,,False,0.96
796,299 to 5k. Baby gains but fuck NVDA,1.743210e+09,1jmamu6,False,Gain,False,t3_1jmamu6,78,False,/r/wallstreetbets/comments/1jmamu6/299_to_5k_b...,,False,0.92


## Part 3: Collecting Comments from Reddit

### Creating a database of reddit threads

In [20]:
# Use the same methofology whih we used to collect submissions, but we'll add a parent submission id. and parent comment id.
# Since the comment section can be very deep, we'll limit comments to a breadth of 10.
# This may still be a lot more comments than we need for larger discussions.
def extract_comments_from_submission(submission_id: str):
    try:
        submission = reddit.submission(id=submission_id)
        submission.comments.replace_more(limit=10)  # Limit to 10 levels of comments
        comments = []

        for comment in submission.comments.list():
            if isinstance(comment, praw.models.MoreComments):
                continue

            # NOTE: It looks like the top comment may be a user report. We'll ignore if is has certain text.
            SKIPTEXT = '**User Report**'
            if SKIPTEXT in comment.body:
                continue

            # Append the comment data to the list
            comments.append({
                'parent_post_id': submission_id,
                'parent_comment_id': comment.parent_id,
                'comment_id': comment.id,
                'author': str(comment.author),
                'created_utc': comment.created_utc,
                'score': comment.score,
                'body': comment.body
            })

        return comments

    except Exception as e:
        # Get the HTTP error code if available
        if hasattr(e, 'response') and e.response is not None:
            error_code = e.response.status_code
            print(f"HTTP Error {error_code} while fetching comments for submission {submission_id}")
        else:
            error_code = None

        # Print the an erroor message and return nothing.
        print(f"Error fetching comments for submission {submission_id}: {e}")
        return []



In [21]:
# Show the results from one submission's comments
submission_id = submission_collection.iloc[0]['id']

# How many actual comments are there for this submission?
submission = reddit.submission(id=submission_id)
print(f"Submission ID: {submission_id}")
print(f"Title: {submission.title}")
print(f"Number of comments: {submission.num_comments}")

# Get the comments for the submission
results = extract_comments_from_submission(submission_id)

# Create a dataframe of the comments
comments_df = pd.DataFrame(results)

# Display the comments dataframe
display(comments_df)

Submission ID: 1k0t4jk
Title: Nivea Along
Number of comments: 6


,parent_post_id,parent_comment_id,comment_id,author,created_utc,score,body
0,1k0t4jk,t3_1k0t4jk,mngn956,Alert_Barber_3105,1.744832e+09,6,The skin cream?
1,1k0t4jk,t3_1k0t4jk,mngmxdi,chrissurra,1.744832e+09,4,You mean Neveah
2,1k0t4jk,t3_1k0t4jk,mngoka2,Reasonable_Roger,1.744832e+09,1,Psoriasis calls
3,1k0t4jk,t1_mngn956,mngnj1n,Own-Foundation3873,1.744832e+09,1,pow till it creams
4,1k0t4jk,t1_mngmxdi,mngnbpk,Own-Foundation3873,1.744832e+09,1,yesss sir


In [22]:
# Collect the comments for all the submissions.
all_comments = []
for submission in submission_collection['id']:
    comments = extract_comments_from_submission(submission)
    all_comments.extend(comments)
    time.sleep(1)  # Optional delay to avoid rate limits

# Create a python dataframe of these comments.
comments_df = pd.DataFrame.from_records(all_comments)
display(comments_df)

,parent_post_id,parent_comment_id,comment_id,author,created_utc,score,body
0,1k0t4jk,t3_1k0t4jk,mngn956,Alert_Barber_3105,1.744832e+09,6,The skin cream?
1,1k0t4jk,t3_1k0t4jk,mngmxdi,chrissurra,1.744832e+09,4,You mean Neveah
2,1k0t4jk,t3_1k0t4jk,mngoka2,Reasonable_Roger,1.744832e+09,1,Psoriasis calls
3,1k0t4jk,t1_mngn956,mngnj1n,Own-Foundation3873,1.744832e+09,1,pow till it creams
4,1k0t4jk,t1_mngmxdi,mngnbpk,Own-Foundation3873,1.744832e+09,1,yesss sir
...,...,...,...,...,...,...,...
151632,1jm85ur,t1_mkehvqw,mkejgzy,CultureForsaken3762,1.743277e+09,2,The Little Book That Beats the Market - Joel G...
151633,1jm85ur,t1_mka50c0,mkfszsb,Lemonibluff,1.743292e+09,3,Agreed. The fight is to become the EveryGamble...
151634,1jm85ur,t1_mkejgzy,mkel24x,Stamperdoodle1,1.743277e+09,1,"amazing recommendation, thank you so much!"
151635,1jm85ur,t1_mkel24x,mkely5t,CultureForsaken3762,1.743278e+09,2,My pleasure. You arent “not smart enough”. Y...


In [23]:
# Save the collected data to parquet format
COMMENT_PARQUET_PATH = './data/wallstreetbets-comment-collection.parquet'

# Create a pyarrow schema for the comment data
comment_schema = pyarrow.schema([
    ('parent_post_id', pyarrow.string()),
    ('parent_comment_id', pyarrow.string()),
    ('comment_id', pyarrow.string()),
    ('author', pyarrow.string()),
    ('created_utc', pyarrow.float64()),
    ('score', pyarrow.int64()),
    ('body', pyarrow.string())
])

In [24]:
# Write the comments to parquet file. If it exists, append to it.
if not os.path.exists(COMMENT_PARQUET_PATH):
    comments_df.to_parquet(COMMENT_PARQUET_PATH, engine='pyarrow', schema=comment_schema)
else:
    old_parquet = pd.read_parquet(COMMENT_PARQUET_PATH, engine='pyarrow', schema=comment_schema)
    new_parquet = pd.concat([old_parquet, comments_df])
    new_parquet = new_parquet.drop_duplicates(subset=['parent_post_id','parent_comment_id','author','created_utc'], keep='last').reset_index(drop=True)
    new_parquet.to_parquet(COMMENT_PARQUET_PATH, engine='pyarrow', schema=comment_schema)